In [3]:
import pandas as pd
import os
import librosa
import keras
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from librosa.feature import mfcc
from librosa.display import specshow

from keras.layers import TimeDistributed
from keras.layers import Conv1D, MaxPool1D, Flatten, Dropout, Dense
from keras.models import Sequential
from keras.optimizers import Adam, SGD
from keras.activations import softmax, relu, tanh
import utils

import sklearn as skl

Using TensorFlow backend.


In [4]:
from IPython import display

In [14]:
metadata_path = 'fma_metadata'
data_path = 'fma_small'

In [15]:
features = utils.load(metadata_path+'/features.csv')
#tracks = utils.load(metadata_path+'/tracks.csv')
echonest = utils.load(metadata_path+'/echonest.csv')
genres = utils.load(metadata_path+'/genres.csv')

In [16]:
tracks = utils.load(metadata_path+'/tracks.csv')

In [17]:

np.testing.assert_array_equal(features.index, tracks.index)
assert echonest.index.isin(tracks.index).all()

tracks.shape, genres.shape, features.shape, echonest.shape

((106574, 52), (163, 4), (106574, 518), (13129, 249))

In [18]:
tracks.head()

album                                                          \
         comments        date_created date_released engineer favorites id   
track_id                                                                    
2               0 2008-11-26 01:44:45    2009-01-05      NaN         4  1   
3               0 2008-11-26 01:44:45    2009-01-05      NaN         4  1   
5               0 2008-11-26 01:44:45    2009-01-05      NaN         4  1   
10              0 2008-11-26 01:45:08    2008-02-06      NaN         4  6   
20              0 2008-11-26 01:45:05    2009-01-06      NaN         2  4   

                                                                           \
                                        information listens producer tags   
track_id                                                                    
2                                           <p></p>    6073      NaN   []   
3                                           <p></p>    6073      NaN   []   
5                                           <p></p>    6073      NaN   []   
10                                              NaN   47632      NaN   []   
20        <p> "spiritual songs" from Nicky Cook</p>    2710      NaN   []   

          ...       track                         \
          ... information interest language_code   
track_id  ...                                      
2         ...         NaN     4656            en   
3         ...         NaN     1470            en   
5         ...         NaN     1933            en   
10        ...         NaN    54881            en   
20        ...         NaN      978            en   

                                                                              \
                                                    license listens lyricist   
track_id                                                                       
2         Attribution-NonCommercial-ShareAlike 3.0 Inter...    1293      NaN   
3         Attribution-NonCommercial-ShareAlike 3.0 Inter...     514      NaN   
5         Attribution-NonCommercial-ShareAlike 3.0 Inter...    1151      NaN   
10        Attribution-NonCommercial-NoDerivatives (aka M...   50135      NaN   
20        Attribution-NonCommercial-NoDerivatives (aka M...     361      NaN   

                                                 
         number publisher tags            title  
track_id                                         
2             3       NaN   []             Food  
3             4       NaN   []     Electric Ave  
5             6       NaN   []       This World  
10            1       NaN   []          Freeway  
20            3       NaN   []  Spiritual Level  

[5 rows x 52 columns]

In [19]:
features.head()

feature    chroma_cens                                                    \
statistics    kurtosis                                                     
number              01        02        03        04        05        06   
track_id                                                                   
2             7.180653  5.230309  0.249321  1.347620  1.482478  0.531371   
3             1.888963  0.760539  0.345297  2.295201  1.654031  0.067592   
5             0.527563 -0.077654 -0.279610  0.685883  1.937570  0.880839   
10            3.702245 -0.291193  2.196742 -0.234449  1.367364  0.998411   
20           -0.193837 -0.198527  0.201546  0.258556  0.775204  0.084794   

feature                                             ...   tonnetz            \
statistics                                          ...       std             
number            07        08        09        10  ...        04        05   
track_id                                            ...                       
2           1.481593  2.691455  0.866868  1.341231  ...  0.054125  0.012226   
3           1.366848  1.054094  0.108103  0.619185  ...  0.063831  0.014212   
5          -0.923192 -0.927232  0.666617  1.038546  ...  0.040730  0.012691   
10          1.770694  1.604566  0.521217  1.982386  ...  0.074358  0.017952   
20         -0.289294 -0.816410  0.043851 -0.804761  ...  0.095003  0.022492   

feature                     zcr                                          \
statistics             kurtosis       max      mean    median       min   
number            06         01        01        01        01        01   
track_id                                                                  
2           0.012111   5.758890  0.459473  0.085629  0.071289  0.000000   
3           0.017740   2.824694  0.466309  0.084578  0.063965  0.000000   
5           0.014759   6.808415  0.375000  0.053114  0.041504  0.000000   
10          0.013921  21.434212  0.452148  0.077515  0.071777  0.000000   
20          0.021355  16.669037  0.469727  0.047225  0.040039  0.000977   

feature                         
statistics      skew       std  
number            01        01  
track_id                        
2           2.089872  0.061448  
3           1.716724  0.069330  
5           2.193303  0.044861  
10          3.542325  0.040800  
20          3.189831  0.030993  

[5 rows x 518 columns]

In [20]:
echonest.head()

echonest                                                    \
         audio_features                                                     
           acousticness danceability    energy instrumentalness  liveness   
track_id                                                                    
2              0.416675     0.675894  0.634476         0.010628  0.177647   
3              0.374408     0.528643  0.817461         0.001851  0.105880   
5              0.043567     0.745566  0.701470         0.000697  0.373143   
10             0.951670     0.658179  0.924525         0.965427  0.115474   
134            0.452217     0.513238  0.560410         0.019443  0.096567   

                                                                        ...  \
                                           metadata                     ...   
         speechiness    tempo   valence  album_date         album_name  ...   
track_id                                                                ...   
2           0.159310  165.922  0.576661         NaN                NaN  ...   
3           0.461818  126.957  0.269240         NaN                NaN  ...   
5           0.124595  100.260  0.621661         NaN                NaN  ...   
10          0.032985  111.562  0.963590  2008-03-11  Constant Hitmaker  ...   
134         0.525519  114.290  0.894072         NaN                NaN  ...   

                                                                              \
         temporal_features                                                     
                       214        215       216       217       218      219   
track_id                                                                       
2                -1.992303   6.805694  0.233070  0.192880  0.027455  0.06408   
3                -1.582331   8.889308  0.258464  0.220905  0.081368  0.06413   
5                -2.288358  11.527109  0.256821  0.237820  0.060122  0.06014   
10               -3.662988  21.508228  0.283352  0.267070  0.125704  0.08082   
134              -1.452696   2.356398  0.234686  0.199550  0.149332  0.06440   

                                                     
                                                     
               220       221        222         223  
track_id                                             
2          3.67696   3.61288  13.316690  262.929749  
3          6.08277   6.01864  16.673548  325.581085  
5          5.92649   5.86635  16.013849  356.755737  
10         8.41401   8.33319  21.317064  483.403809  
134       11.26707  11.20267  26.454180  751.147705  

[5 rows x 249 columns]

In [21]:
genres.head()

,#tracks,parent,title,top_level
genre_id,,,,
1,8693,38,Avant-Garde,38
2,5271,0,International,2
3,1752,0,Blues,3
4,4126,0,Jazz,4
5,4106,0,Classical,5


In [22]:
genres

,#tracks,parent,title,top_level
genre_id,,,,
1,8693,38,Avant-Garde,38
2,5271,0,International,2
3,1752,0,Blues,3
4,4126,0,Jazz,4
5,4106,0,Classical,5
...,...,...,...,...
1032,60,102,Turkish,2
1060,30,46,Tango,2
1156,26,130,Fado,2


In [23]:
len(np.unique(genres.top_level))

16

In [24]:
learning_rate = 0.001

In [25]:
dict_genres = {'Electronic':0, 'Experimental':1, 'Folk':2, 'Hip-Hop':3, 
               'Instrumental':4,'International':5, 'Pop' :6, 'Rock': 7  }


In [26]:
small = tracks['set', 'subset'] <= 'small'

In [27]:
small

track_id
2          True
3         False
5          True
10         True
20        False
          ...  
155316    False
155317    False
155318    False
155319    False
155320    False
Name: (set, subset), Length: 106574, dtype: bool

In [28]:
train = tracks['set','split'] == 'training'
test = tracks['set','split'] == 'test'
val = tracks['set','split'] == 'val'

In [29]:
X_train = features.loc[train & small, 'mfcc']
X_test = features.loc[test & small, 'mfcc']
X_val = features.loc[val & small, 'mfcc']

In [30]:
y_train = tracks.loc[train & small, ('track', 'genre_top')]
y_test = tracks.loc[test & small, ('track', 'genre_top')]
y_val = tracks.loc[val & small, ('track', 'genre_top')]

In [31]:
len(np.unique(genres.top_level))

16

In [32]:
tracks

album                                                              \
         comments        date_created date_released engineer favorites     id   
track_id                                                                        
2               0 2008-11-26 01:44:45    2009-01-05      NaN         4      1   
3               0 2008-11-26 01:44:45    2009-01-05      NaN         4      1   
5               0 2008-11-26 01:44:45    2009-01-05      NaN         4      1   
10              0 2008-11-26 01:45:08    2008-02-06      NaN         4      6   
20              0 2008-11-26 01:45:05    2009-01-06      NaN         2      4   
...           ...                 ...           ...      ...       ...    ...   
155316          0 2017-03-30 15:20:35    2017-02-17      NaN         0  22940   
155317          0 2017-03-30 15:20:35    2017-02-17      NaN         0  22940   
155318          0 2017-03-30 15:20:35    2017-02-17      NaN         0  22940   
155319          0 2017-03-30 15:20:35    2017-02-17      NaN         0  22940   
155320          0 2017-03-26 16:22:18    2017-03-26      NaN         1  22906   

                                                                     \
                                                information listens   
track_id                                                              
2                                                   <p></p>    6073   
3                                                   <p></p>    6073   
5                                                   <p></p>    6073   
10                                                      NaN   47632   
20                <p> "spiritual songs" from Nicky Cook</p>    2710   
...                                                     ...     ...   
155316    <p>A live performance at Monty Hall on Feb 17,...    1506   
155317    <p>A live performance at Monty Hall on Feb 17,...    1506   
155318    <p>A live performance at Monty Hall on Feb 17,...    1506   
155319    <p>A live performance at Monty Hall on Feb 17,...    1506   
155320                                                  NaN    7481   

                                                                          ...  \
             producer                                               tags  ...   
track_id                                                                  ...   
2                 NaN                                                 []  ...   
3                 NaN                                                 []  ...   
5                 NaN                                                 []  ...   
10                NaN                                                 []  ...   
20                NaN                                                 []  ...   
...               ...                                                ...  ...   
155316    Monty Hall                                                  []  ...   
155317    Monty Hall                                                  []  ...   
155318    Monty Hall                                                  []  ...   
155319    Monty Hall                                                  []  ...   
155320            NaN  [ballad, epic, rockabilly, curse, hex, hard ro...  ...   

               track                         \
         information interest language_code   
track_id                                      
2                NaN     4656            en   
3                NaN     1470            en   
5                NaN     1933            en   
10               NaN    54881            en   
20               NaN      978            en   
...              ...      ...           ...   
155316           NaN      122           NaN   
155317           NaN      194           NaN   
155318           NaN      214           NaN   
155319           NaN      336           NaN   
155320           NaN      972           NaN   

                                                                              \
        

In [205]:
paths = pd.read_csv('paths.csv', dtype='str')

In [206]:
paths.head()

,root_dir,filename
0,104,104063
1,104,104779
2,104,104284
3,104,104278
4,104,104064


In [336]:
def create_sets():
    X = []
    i=0
    x=[]
    y = []
    for d in paths.values[:5]:
        i+=1
        print(d)
        fn = librosa.load(os.path.join(data_path, str(d[0]))+'/'+str(d[1])+'.mp3')
        mfcs = librosa.feature.mfcc(fn[0])
        print(mfcs)
        if i==1:
            mfcs_r = mfcs.reshape(-1, mfcs.shape[0], mfcs.shape[1])
            X = mfcs_r.copy()
        else:
            np.vstack((X, mfcs_r))
        y.append(y_train.get(int(d[1])))
        print(y)
        X = np.vstack((X, mfcs_r))
    return X, y

In [338]:
X, y = create_sets()
#np.reshape()

['104' '104063']


/home/dexter/anaconda3/lib/python3.6/site-packages/librosa/core/audio.py:146: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


[[-3.83210724e+02 -3.19389740e+02 -2.87044525e+02 ... -1.27689819e+02
  -1.67805420e+02 -1.93448639e+02]
 [ 1.17472275e+02  1.69759369e+02  1.88922729e+02 ...  4.08468590e+01
   6.26655502e+01  7.01413803e+01]
 [ 3.07044373e+01  3.46233826e+01  3.51768417e+01 ...  4.22330933e+01
   3.83826752e+01  3.09761772e+01]
 ...
 [-9.35981083e+00 -7.11590099e+00 -3.65561461e+00 ...  1.62628956e+01
   2.18400421e+01  2.42012329e+01]
 [-4.81917000e+00  1.73145843e+00  1.23919916e+00 ...  2.38578844e+00
   8.03371239e+00  7.72657919e+00]
 [ 9.66041982e-01  1.18436980e+00  1.41247451e-01 ...  1.11482024e+00
   1.72722459e+00 -6.37300134e-01]]
['Rock']
['104' '104779']


/home/dexter/anaconda3/lib/python3.6/site-packages/librosa/core/audio.py:146: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


[[-258.78894    -213.50655    -162.74886    ... -199.1468
  -199.87851    -176.18562   ]
 [ 176.82123     163.37134     137.14511    ...   97.70882
   105.53371      89.2008    ]
 [  48.244156     -0.31717134  -11.516499   ...   18.698692
    34.85655      38.172707  ]
 ...
 [  -7.232548     -0.87324154   -0.9571284  ...    7.392023
     4.832102     11.139341  ]
 [  12.092382      8.949335     -0.43519217 ...   -4.982649
     1.2126809    -0.89253646]
 [   4.046933     -0.9530624    -4.383523   ...    5.2060003
    -6.7787285    -8.859343  ]]
['Rock', None]
['104' '104284']


...

[[-420.83224   -201.90164   -127.34894   ...   29.439589   -29.596819
   -63.542515 ]
 [  76.12117    100.16031     95.93286   ...    8.98914     27.983242
    46.935856 ]
 [  43.170418    44.624504    40.236225  ...   29.820663    26.321198
    39.134903 ]
 ...
 [  -2.415881    -1.952828     1.2916702 ...    8.077227    11.3324
     6.6064906]
 [  -6.9624715   -4.3621707   -3.982636  ...   -7.2947736    5.793949
     7.9043927]
 [  -9.609051     3.9626982    4.2463713 ...    3.8673487   13.116399
    16.005144 ]]
['Rock', None, 'Electronic']
['104' '104278']


/home/dexter/anaconda3/lib/python3.6/site-packages/librosa/core/audio.py:146: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


[[-187.7583    -152.30728   -146.68506   ... -250.03354   -239.91461
  -222.2049   ]
 [ 116.19923    109.696465   104.97185   ...   79.75784     85.74104
    81.822174 ]
 [  26.247587    23.384382    18.748184  ...   19.168581    10.776802
     3.927683 ]
 ...
 [   0.930201     2.2403069    1.6038883 ...  -17.260853   -16.616867
   -17.48161  ]
 [   4.653384     5.439618     4.4495625 ...    7.096718     6.349988
     6.8379993]
 [  12.218077    11.090695     6.326832  ...   18.60473     20.366196
    21.634417 ]]
['Rock', None, 'Electronic', 'Electronic']
['104' '104064']


/home/dexter/anaconda3/lib/python3.6/site-packages/librosa/core/audio.py:146: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


[[-152.31502     -27.034119     13.509284   ...  -42.623447
   -32.40346     -25.753403  ]
 [  77.46075      90.16641      87.55169    ...   69.55633
    72.40141      73.33661   ]
 [ -44.362965    -37.162403    -32.898766   ...   45.678257
    62.700066     67.75954   ]
 ...
 [  -5.7111397     0.8322245     7.1959686  ...   -2.641518
    -5.4960275    -9.944369  ]
 [   5.719557      5.2845936     4.781291   ...   -0.34231094
    -5.465396     -9.959821  ]
 [  -2.8613796    -5.092187     -6.008932   ...    1.0892565
     3.1481562     2.3421636 ]]
['Rock', None, 'Electronic', 'Electronic', 'Pop']


In [339]:
X.shape

(6, 20, 1293)

In [340]:
y

['Rock', None, 'Electronic', 'Electronic', 'Pop']

In [308]:
y.append(y_train.get(104063))

In [89]:
mfcc = librosa.feature.mfcc(f[0])

In [99]:
mfcc.shape

(1, 20, 1291)

In [92]:
qw = []
qw.append(mfcc)

In [98]:
mfcc = mfcc.reshape(-1,mfcc.shape[0], mfcc.shape[1])

In [165]:
train

track_id
2          True
3          True
5          True
10         True
20         True
          ...  
155316     True
155317     True
155318     True
155319     True
155320    False
Name: (set, split), Length: 106574, dtype: bool

In [176]:
y_train.get(2)

'Hip-Hop'

In [322]:
X = []
i=0
x=[]
y = []
for d in paths.values[:5]:
    i+=1
    print(d[1])
    print(y_train.get(d[1]))
    fn = librosa.load(os.path.join(data_path, str(d[0]))+'/'+str(d[1])+'.mp3')
    mfcs = librosa.feature.mfcc(fn[0])
    print(mfcs)
    if i==1:
        mfcs_r = mfcs.reshape(-1, mfcs.shape[0], mfcs.shape[1])
        X = mfcs_r.copy()
    else:
        np.vstack((X, mfcs_r))
    cl = y_train.get(int(d[1])
    print(cl)
    y.append(cl)
    print(y)
    X = np.vstack((X, mfcs_r))

104063
None


/home/dexter/anaconda3/lib/python3.6/site-packages/librosa/core/audio.py:146: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


[[-3.83210724e+02 -3.19389740e+02 -2.87044525e+02 ... -1.27689819e+02
  -1.67805420e+02 -1.93448639e+02]
 [ 1.17472275e+02  1.69759369e+02  1.88922729e+02 ...  4.08468590e+01
   6.26655502e+01  7.01413803e+01]
 [ 3.07044373e+01  3.46233826e+01  3.51768417e+01 ...  4.22330933e+01
   3.83826752e+01  3.09761772e+01]
 ...
 [-9.35981083e+00 -7.11590099e+00 -3.65561461e+00 ...  1.62628956e+01
   2.18400421e+01  2.42012329e+01]
 [-4.81917000e+00  1.73145843e+00  1.23919916e+00 ...  2.38578844e+00
   8.03371239e+00  7.72657919e+00]
 [ 9.66041982e-01  1.18436980e+00  1.41247451e-01 ...  1.11482024e+00
   1.72722459e+00 -6.37300134e-01]]
None
[None]
104779
None


/home/dexter/anaconda3/lib/python3.6/site-packages/librosa/core/audio.py:146: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


[[-258.78894    -213.50655    -162.74886    ... -199.1468
  -199.87851    -176.18562   ]
 [ 176.82123     163.37134     137.14511    ...   97.70882
   105.53371      89.2008    ]
 [  48.244156     -0.31717134  -11.516499   ...   18.698692
    34.85655      38.172707  ]
 ...
 [  -7.232548     -0.87324154   -0.9571284  ...    7.392023
     4.832102     11.139341  ]
 [  12.092382      8.949335     -0.43519217 ...   -4.982649
     1.2126809    -0.89253646]
 [   4.046933     -0.9530624    -4.383523   ...    5.2060003
    -6.7787285    -8.859343  ]]
None
[None, None]
104284
None


...

[[-420.83224   -201.90164   -127.34894   ...   29.439589   -29.596819
   -63.542515 ]
 [  76.12117    100.16031     95.93286   ...    8.98914     27.983242
    46.935856 ]
 [  43.170418    44.624504    40.236225  ...   29.820663    26.321198
    39.134903 ]
 ...
 [  -2.415881    -1.952828     1.2916702 ...    8.077227    11.3324
     6.6064906]
 [  -6.9624715   -4.3621707   -3.982636  ...   -7.2947736    5.793949
     7.9043927]
 [  -9.609051     3.9626982    4.2463713 ...    3.8673487   13.116399
    16.005144 ]]
None
[None, None, None]
104278
None


/home/dexter/anaconda3/lib/python3.6/site-packages/librosa/core/audio.py:146: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


[[-187.7583    -152.30728   -146.68506   ... -250.03354   -239.91461
  -222.2049   ]
 [ 116.19923    109.696465   104.97185   ...   79.75784     85.74104
    81.822174 ]
 [  26.247587    23.384382    18.748184  ...   19.168581    10.776802
     3.927683 ]
 ...
 [   0.930201     2.2403069    1.6038883 ...  -17.260853   -16.616867
   -17.48161  ]
 [   4.653384     5.439618     4.4495625 ...    7.096718     6.349988
     6.8379993]
 [  12.218077    11.090695     6.326832  ...   18.60473     20.366196
    21.634417 ]]
None
[None, None, None, None]
104064
None


/home/dexter/anaconda3/lib/python3.6/site-packages/librosa/core/audio.py:146: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


[[-152.31502     -27.034119     13.509284   ...  -42.623447
   -32.40346     -25.753403  ]
 [  77.46075      90.16641      87.55169    ...   69.55633
    72.40141      73.33661   ]
 [ -44.362965    -37.162403    -32.898766   ...   45.678257
    62.700066     67.75954   ]
 ...
 [  -5.7111397     0.8322245     7.1959686  ...   -2.641518
    -5.4960275    -9.944369  ]
 [   5.719557      5.2845936     4.781291   ...   -0.34231094
    -5.465396     -9.959821  ]
 [  -2.8613796    -5.092187     -6.008932   ...    1.0892565
     3.1481562     2.3421636 ]]
None
[None, None, None, None, None]


In [335]:
y_train.get(int(d[1]))

'Pop'

In [332]:
y_train.get(104064)

'Pop'

In [334]:
type(d[1])

str

In [274]:
mfcs = librosa.feature.mfcc(fn[0])

In [275]:
mfcs.shape

(20, 1291)

In [276]:
mfcs_r = mfcs.reshape(-1, mfcs.shape[0], mfcs.shape[1])

In [285]:
mfcs_r.shape

(1, 20, 1291)

(2, 20, 1291)

In [ ]:
mf